## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

c:\abc\envs\ami4\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [3]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':50,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':32,
    'SEED':777
}

## Fixed RandomSeed

In [15]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

## Data Pre-processing

In [16]:
df = pd.read_csv('./train.csv')

In [17]:
train_len = int(len(df) * 0.8)
train_df = df.iloc[:train_len]

remaining_df = df.iloc[train_len:]
val_len = int(len(remaining_df) * 0.5)

val_df = remaining_df.iloc[:val_len]
test_df = remaining_df.iloc[val_len:]

In [18]:
train_label_vec = train_df.iloc[:, 2:].values.astype(np.float32)
val_label_vec = val_df.iloc[:, 2:].values.astype(np.float32)
test_label_vec = test_df.iloc[:, 2:].values.astype(np.float32)

In [19]:
CFG['label_size'] = train_label_vec.shape[1]

## CustomDataset

In [20]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [21]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [22]:
train_dataset = CustomDataset(train_df['path'].values, train_label_vec, train_transform)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['path'].values, val_label_vec, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

test_dataset = CustomDataset(test_df['path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [24]:
class Efficientnet(nn.Module):
    def __init__(self, gene_size=CFG['label_size'], dropout_rate=0.5):
        super(Efficientnet, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.backbone.classifier = nn.Identity() 
        self.regressor = nn.Sequential(
            nn.SiLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(1280, gene_size) 
        )
        
    def forward(self, x):
        x = self.backbone(x)  
        x = self.regressor(x) 
        return x

## Train

In [25]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    
    best_loss = float('inf')
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_loss)
            
        if best_loss > _val_loss:
            best_loss = _val_loss
            best_model = model
    
    return best_model

In [26]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
    
    return _val_loss

In [27]:
def test(model, criterion, test_loader, device):
    model.eval()
    test_loss = []
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(test_loader)):
            labels = labels.to(device)

            pred = model(imgs)

            loss = criterion(pred, labels)
            test_loss.append(loss.item())

            all_preds.append(pred.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            
        _test_loss = np.mean(test_loss)

    print(f'Test Loss : [{_test_loss:.5f}]')
    return _test_loss, all_preds, all_labels

## Run!!

In [28]:
model1 = Efficientnet()
optimizer1 = torch.optim.Adam(params=model1.parameters(), lr=CFG["LEARNING_RATE"])
scheduler1 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer1, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
trained_model1 = train(model1, optimizer1, train_loader, val_loader, scheduler1, device)

model2 = Efficientnet()
optimizer2 = torch.optim.Adam(params=model2.parameters(), lr=CFG["LEARNING_RATE"] * 0.8)
scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer2, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
trained_model2 = train(model2, optimizer2, train_loader, val_loader, scheduler2, device)

model3 = Efficientnet()
optimizer3 = torch.optim.Adam(params=model3.parameters(), lr=CFG["LEARNING_RATE"] * 1.2)
scheduler3 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer3, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
trained_model3 = train(model3, optimizer3, train_loader, val_loader, scheduler3, device)

model4 = Efficientnet()
optimizer4 = torch.optim.Adam(params=model4.parameters(), lr=CFG["LEARNING_RATE"] * 0.9)
scheduler4 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer4, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
trained_model4 = train(model4, optimizer4, train_loader, val_loader, scheduler4, device)

model5 = Efficientnet()
optimizer5 = torch.optim.Adam(params=model5.parameters(), lr=CFG["LEARNING_RATE"] * 1.1)
scheduler5 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer5, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
trained_model5 = train(model5, optimizer5, train_loader, val_loader, scheduler5, device)

100%|██████████| 22/22 [00:03<00:00,  6.98it/s]


Epoch [1], Train Loss : [0.05675] Val Loss : [0.04775]


100%|██████████| 22/22 [00:02<00:00,  7.63it/s]


Epoch [2], Train Loss : [0.04818] Val Loss : [0.04703]


100%|██████████| 22/22 [00:02<00:00,  7.59it/s]


...

Epoch [48], Train Loss : [0.04374] Val Loss : [0.04578]


100%|██████████| 22/22 [00:03<00:00,  6.47it/s]


Epoch [49], Train Loss : [0.04375] Val Loss : [0.04576]


100%|██████████| 22/22 [00:03<00:00,  6.47it/s]

Epoch [50], Train Loss : [0.04374] Val Loss : [0.04577]


## Inference

In [29]:
test = pd.read_csv('./test.csv')

In [30]:
test_dataset = CustomDataset(test['path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [31]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(test_loader):
            imgs = imgs.to(device).float()
            pred = model(imgs)
            
            preds.append(pred.detach().cpu())
    
    preds = torch.cat(preds).numpy()

    return preds

## Submission

In [32]:
criterion = nn.MSELoss()

val_loss1 = validation(trained_model1, criterion, val_loader, device)
val_loss2 = validation(trained_model2, criterion, val_loader, device)
val_loss3 = validation(trained_model3, criterion, val_loader, device)
val_loss4 = validation(trained_model4, criterion, val_loader, device)
val_loss5 = validation(trained_model5, criterion, val_loader, device)

losses = [val_loss1, val_loss2, val_loss3, val_loss4, val_loss5]
inverse_losses = [1 / loss for loss in losses]

total_inverse_loss = sum(inverse_losses)
weights = [loss / total_inverse_loss for loss in inverse_losses]

print(f"Calculated weights based on validation losses: {weights}")

preds1 = inference(trained_model1, test_loader, device)
preds2 = inference(trained_model2, test_loader, device)
preds3 = inference(trained_model3, test_loader, device)
preds4 = inference(trained_model4, test_loader, device)
preds5 = inference(trained_model5, test_loader, device)

ensemble_preds = (preds1 * weights[0] + preds2 * weights[1] + preds3 * weights[2] +
                  preds4 * weights[3] + preds5 * weights[4])

submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = ensemble_preds.astype(np.float32)
submit.to_csv('./efficientnetb0_2t.csv', index=False)

print(f"Weighted Ensemble Predictions saved to efficientnet_weighted_ensemble_submit.csv")

100%|██████████| 22/22 [00:03<00:00,  6.28it/s]


Calculated weights based on validation losses: [0.20024229245002012, 0.2002900797601681, 0.20012045162047518, 0.1995789098247095, 0.1997682663446271]


100%|██████████| 72/72 [00:07<00:00,  9.56it/s]


Weighted Ensemble Predictions saved to efficientnet_weighted_ensemble_submit.csv
